In [2]:
import numpy as np
#from a_simple_1_MPS import split_truncate_theta
#import a_simple_1_MPS
#import a_simple_2_model
#from a_simple_4_DMRG import *
from DMRG_101 import *
import scipy.sparse
import scipy.sparse.linalg.eigen.arpack as arp

import datetime

# import matplotlib server friendly

#from matplotlib import pyplot as plt
import matplotlib
#matplotlib.use('Agg')
import pylab as plt

#miscellaneous

from collections import Counter

We are exemplarily looking at the transverse field Ising model
$$
H = -\sum_i \sigma_i^x \sigma_{i+1}^x - g \sum_i \sigma_i^z
$$
with magnetization $m = \sum_i \sigma_i^x$ in x-direction as an oder parameter for the 2nd order phase transition. for a finite system size this magnetization looks like this:

![phase diagram of the TFIM in terms of the magnetization m](train_and_test.png)

Imagine you are not aware of this order parameter but due to symmetry considerations know that there are two distinct phases for very small g and very big g respectively. The idea is to create training data *deep in the phase* for which we are sure of the phase such that we can label them accordingly.  
The following generates this training data. In order to get good results we have to compute many of these states of the order $\mathcal{O}(10^4)$. We choose $NN = 10.000$ for each phase for $g\in[0,0.4]$ and $g\in[1.6,2]$.

In [9]:
NN = 10


In [16]:
def create_training(L,chi_max,NN):
    
    ### g's deep in the phase
    gs0 = np.linspace(0.1,0.4,NN)
    gs1 = np.linspace(1.6,2,NN)
    
    ### Data Creation

    ######################################
    ### Hyper Parameters #################
    ######################################
    hyper_dict = {'L' : L,'conf' : 1e-12, 'chi_max' : chi_max, "h" : 0. ,'eps' : 1.e-14,'initial_state' : 'random'}

    ######################################
    ### DMRG Definition ##################
    ######################################

    def bond_dim(psi):
        return max(max([psi.Ms[i].shape for i in range(len(psi.Ms))]))

    def dmrg_psi(L,g,h,conf=1e-13,chi_max=30,eps = 1e-17,min_sweeps = 2):
        M = Ising_MPO(L=L,d=2, J=1., h=g,)
        psi = random_MPS(L=L,d=2,chi_max=chi_max)
        # always starts from big random state because single site dmrg
        eng = DMRG(MPS = psi, MPO = M, chi_max=chi_max, eps=eps)
        relE = 3*conf
        counter=0
        while relE>conf:
            E1 = eng.es
            eng.sweep()
            E2 = eng.es
            relE = abs((E2-E1) / E2)
            counter +=1
            #E = np.mean(psi.bond_expectation_value(M.H_bonds))
            #print("sweep {i:2d}: E/L = {E:.13f}".format(i=i + 1, E=E))
        #print("final bond dimensions: ", psi.get_chi())
        #print("correlation length:", psi.correlation_length())
        mag = np.sum(psi.all_exp(np.array([[0., 1.], [1., 0.]])))/psi.L
        #bond_dimension = bond_dim(psi)
        print("{counter} sweeps, L = {L} , g={g:.7f} ; E = {e:.13f}, m = {m}, rel err = {relE:.17f}, bond_dim = {bd:d}".format(L=hyper_dict["L"],m=mag,counter=counter,g=g,e=E2,relE=relE,bd=bond_dim(psi)))
        return psi,mag


    ######################################
    ### Create Data ######################
    ######################################

    ### Filenames ###
    train_outfile = 'train_data/training_D-' + str(chis[i]) + "_L-" + str(hyper_dict["L"]) + "_NN-" + str(NN) + '.npz'
    train_outfile_full = 'train_data/training_D-' + str(chis[i]) + "_L-" + str(hyper_dict["L"]) + "_NN-" + str(NN) + '_full-psis.npz'

    import datetime
    starttime = datetime.datetime.now()
    print(starttime)
    
    ### Calculating states in resepctive phases ###
    
    # Phase 0
    phase0 = [None]*len(gs0)
    mags0=[None]*len(gs0)
    for i in range(len(gs0)):
        g = gs0[i]
        phase0[i],mags0[i] = dmrg_psi(L=hyper_dict["L"],g=g,h=hyper_dict['h'],chi_max=hyper_dict['chi_max'],conf=hyper_dict['conf'],eps = hyper_dict['eps'])

    print('phase 0 complete')
    print(datetime.datetime.now())

    
    # Phase 1
    print('Starting with phase 1, i.e. g=1.5 .. 2')
    phase1 = [None]*len(gs1)
    mags1=[None]*len(gs1)
    for i in range(len(gs1)):
        g = gs1[i]
        phase1[i],mags1[i] = dmrg_psi(L=hyper_dict["L"],g=g,h=hyper_dict['h'],chi_max=hyper_dict['chi_max'],conf=hyper_dict['conf'],eps = hyper_dict['eps'])
    print('phase 1 complete')


    ### Saving the full states 
    np.savez(train_outfile_full,psi_0 = phase0,gs_0 = gs0,psi_1 = phase1,gs_1 = gs1,mags0 = mags0, mags1 = mags1,hyper_dict = hyper_dict)

    def makebig(B,bond_dim=30,d=2):
        """artifically increases the dimension of a matrix of bond dimension < 30"""
        M = B.transpose([0,2,1]) #changing from shape 30,2,30 to 30,30,2
        dummy = np.zeros([bond_dim,bond_dim,d])
        chi = M.shape[0]
        chi2 = M.shape[1]
        dummy[0:chi,0:chi2,:] = M
        return dummy

    def prepare_phase(phase):
        """
        Input:: phase: list of MPS for a phase
        Output:: List of matrices in the shape (30,30,2)
        """
        output = []
        for i in range(len(phase)):
            # There are NN ~ 10k samples in every phase, 
            # not sure if this for-loop is such a good idea,
            # might have to change how the data is generated in the first place
            psi = phase[i]
            # take matrix in the middle of the chain
            L2 = int(psi.L/2) 
            B = np.real_if_close(psi.Ms[L2])
            newB = makebig(B) # outputs (30,30,2)
            output.append(newB)
        return output

    def prepare_data(phase0_psis,phase1_psis):
        list0 = prepare_phase(phase0_psis)
        list1 = prepare_phase(phase1_psis)
        list3 = list0+list1
        return np.stack(list3,axis=0)

    np.savez(train_outfile,training=prepare_data(phase0,phase1),mags0=mags0,mags1=mags1,gs0=gs0,gs1=gs1,hyper_dict = hyper_dict)
    print("saved in " + train_outfile)
    endtime = datetime.datetime.now()
    print(datetime.datetime.now())
    print('execution time {}'.format(endtime-starttime))

Just as an example we calculate some states. The provided set <training.npz> contains the data from states calcualtes for L=64 and chi_max=10 for NN=10k states in each phase.

In [14]:
create_training(L=30,chi_max=10,gs0=gs0,gs1=gs1)

2019-04-08 17:46:41.520897
2 sweeps, L = 30 , g=0.1000000 ; E = -29.0800532662080, m = 0.9984921834629978, rel err = 0.00000000000000330, bond_dim = 7
2 sweeps, L = 30 , g=0.1333333 ; E = -29.1423909194257, m = 0.9973084892384568, rel err = 0.00000000000000146, bond_dim = 9
2 sweeps, L = 30 , g=0.1666667 ; E = -29.2226351864521, m = 0.9957722329078772, rel err = 0.00000000000000231, bond_dim = 10
2 sweeps, L = 30 , g=0.2000000 ; E = -29.3208591515744, m = 0.9938721508875556, rel err = 0.00000000000000145, bond_dim = 10
2 sweeps, L = 30 , g=0.2333333 ; E = -29.4371535043840, m = 0.9915938988740571, rel err = 0.00000000000000290, bond_dim = 10
2 sweeps, L = 30 , g=0.2666667 ; E = -29.5716274160217, m = 0.9889196563511042, rel err = 0.00000000000000060, bond_dim = 10
2 sweeps, L = 30 , g=0.3000000 ; E = -29.7244096314851, m = 0.9858276084553267, rel err = 0.00000000000000048, bond_dim = 10
2 sweeps, L = 30 , g=0.3333333 ; E = -29.8956498119913, m = 0.9822912713099364, rel err = 0.00000000